Objective : classify a playlist name with AutoML

In [1]:
###
# help Idk stuff corner
###

# https://cloud.google.com/natural-language/automl/docs/tutorial
# !pip install google-cloud-automl

# it seems BigQuery ML *is* AutoML so that's grand
# https://cloud.google.com/bigquery-ml/
# docs/reference/standard-sql/bigqueryml-syntax-create-automl

# how to predict once model is created ?
# https://github.com/googleapis/python-automl/blob/main/samples/snippets/translate_predict.py

In [2]:
import os
import json

from google.cloud import bigquery
from google.cloud import storage
from google.cloud import automl

from botocore.exceptions import ClientError

In [3]:
home = os.path.expanduser('~')
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=f"{home}/.config/gcloud/alpine-beacon-336222-c6a1c087e3ef.json"

In [4]:
# more cringe binge
with open("sql/dml/count_playlist_names.sql", "r") as the_file:
    the_qRy = ""
    while True:
        line = the_file.readline()
        if line == "":
            break
        else:
            the_qRy = the_qRy + " " + line

In [5]:
client = bigquery.Client()
project = client.project
dataset_id = "{}.music".format(project)
dataset = bigquery.Dataset(dataset_id)

In [6]:
qry_job = client.query(the_qRy)
results = qry_job.result()
df = results.to_dataframe()

In [7]:
df.head(10)

,count,name
0,78,Country
1,54,Christmas
2,54,Chill
3,48,Rap
4,48,Rock
5,36,chill
6,36,country
7,36,throwback
8,36,ClassicRock
9,30,Workout


In [8]:
# now isolate a table with playlist name in this list

In [9]:
relevants = df.head(20).name.to_list()

In [10]:
relevants

['Country',
 'Christmas',
 'Chill',
 'Rap',
 'Rock',
 'chill',
 'country',
 'throwback',
 'ClassicRock',
 'Workout',
 '����',
 'lit',
 'workout',
 '2017',
 'calm',
 'party',
 'Sleep',
 'Oldies',
 '❤️',
 'november']

In [11]:
# more cringe binge
with open("sql/ddl/yoink_relevants.sql", "r") as the_file:
    the_relevant_qRy = ""
    while True:
        line = the_file.readline()
        if line == "":
            break
        else:
            the_relevant_qRy = the_relevant_qRy + " " + line

In [12]:
# delete table if it exists
tables = client.list_tables(dataset_id)  # Make an API request.

print("Tables contained in '{}':".format(dataset_id))
for table in tables:
    if table.table_id == "relevants": 
        fqn = "{}.{}.{}".format(table.project, table.dataset_id, table.table_id)
        print(fqn)
        print("deleting relevants . . . ")
        client.delete_table(table)
        print("d o n e")

Tables contained in 'alpine-beacon-336222.music':
alpine-beacon-336222.music.relevants
deleting relevants . . . 
d o n e


In [13]:
tables = client.list_tables(dataset_id)  # Make an API request.

print("Tables now contained in '{}':".format(dataset_id))
for table in tables:
    print("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))

Tables now contained in 'alpine-beacon-336222.music':
alpine-beacon-336222.music.slices


In [14]:
try:
    create_table_job = client.query(the_relevant_qRy)
    create_table_job.result()
except ClientError as eek:
    for eRr in create_table.errors:
        print(f"{eRr} \n")

In [15]:
with open("sql/models/classification_model.sql", "r") as the_file:
    model_qRy = ""
    while True:
        line = the_file.readline()
        if line == "":
            break
        else:
            model_qRy = model_qRy + " " + line

print(model_qRy)

 CREATE OR REPLACE MODEL `music.classification_model2`
 OPTIONS
   (model_type='AUTOML_CLASSIFIER',
   input_label_cols=['name']) AS
 SELECT 
     name,
     collaborative,
     artist_name,
     track_name,
     album_name,
     num_edits,
     description,
     num_artists,
     modified_at
 FROM (
 SELECT *
     FROM `alpine-beacon-336222.music.relevants`,
     UNNEST(tracks) as TRK
 )
 LIMIT 1000;


In [16]:
%%time
model_query_job = client.query(model_qRy)
model_results = model_query_job.result()

BadRequest: 400 Failed to calculate avg_string_length since the entries in corresponding column 'description' are all NULLs.

Location: US
Job ID: 05378acc-5e23-4270-a753-6cd2a005b6aa


In [17]:
def list_datasets(project_id):
    """List datasets."""

    client = automl.AutoMlClient()
    # A resource that represents Google Cloud Platform location.
    project_location = f"projects/{project_id}/locations/us-central1"

    # List all the datasets available in the region.
    request = automl.ListDatasetsRequest(parent=project_location, filter="")
    response = client.list_datasets(request=request)

    print("List of datasets:")
    for dataset in response:
        print("Dataset name: {}".format(dataset.name))
        print("Dataset id: {}".format(dataset.name.split("/")[-1]))
        print("Dataset display name: {}".format(dataset.display_name))
        print("Dataset create time: {}".format(dataset.create_time))
        # [END automl_language_sentiment_analysis_list_datasets]
        # [END automl_language_text_classification_list_datasets]
        # [END automl_translate_list_datasets]
        # [END automl_vision_classification_list_datasets]
        # [END automl_vision_object_detection_list_datasets]
        print(
            "Text extraction dataset metadata: {}".format(
                dataset.text_extraction_dataset_metadata
            )
        )
        # [END automl_language_entity_extraction_list_datasets]

        # [START automl_language_sentiment_analysis_list_datasets]
        print(
            "Text sentiment dataset metadata: {}".format(
                dataset.text_sentiment_dataset_metadata
            )
        )
        # [END automl_language_sentiment_analysis_list_datasets]

        # [START automl_language_text_classification_list_datasets]
        print(
            "Text classification dataset metadata: {}".format(
                dataset.text_classification_dataset_metadata
            )
        )
        # [END automl_language_text_classification_list_datasets]

        # [START automl_translate_list_datasets]
        print("Translation dataset metadata:")
        print(
            "\tsource_language_code: {}".format(
                dataset.translation_dataset_metadata.source_language_code
            )
        )
        print(
            "\ttarget_language_code: {}".format(
                dataset.translation_dataset_metadata.target_language_code
            )
        )
        # [END automl_translate_list_datasets]

        # [START automl_vision_classification_list_datasets]
        print(
            "Image classification dataset metadata: {}".format(
                dataset.image_classification_dataset_metadata
            )
        )
        # [END automl_vision_classification_list_datasets]

        # [START automl_vision_object_detection_list_datasets]
        print(
            "Image object detection dataset metadata: {}".format(
                dataset.image_object_detection_dataset_metadata
            )
        )
        # [END automl_vision_object_detection_list_datasets]


In [18]:
project_id = client.project
list_datasets(project_id)

List of datasets:
Dataset name: projects/986744994995/locations/us-central1/datasets/TCN4345301289052143616
Dataset id: TCN4345301289052143616
Dataset display name: music
Dataset create time: 2022-02-26 00:26:22.581011+00:00
Text extraction dataset metadata: 
Text sentiment dataset metadata: 
Text classification dataset metadata: classification_type: MULTICLASS

Translation dataset metadata:
	source_language_code: 
	target_language_code: 
Image classification dataset metadata: 
Image object detection dataset metadata: 
